In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torch.profiler import profile, record_function, ProfilerActivity

In [5]:
composed = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [6]:
root_dir = 'D:\StarsVsGalaxy'
dataset = datasets.ImageFolder(root=root_dir, transform=composed)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

In [8]:
class SvG_CNN(nn.Module):
    def __init__(self):
        super(SvG_CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(50176, 1000)
        self.fc2 = nn.Linear(1000, 1)
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [9]:
model = SvG_CNN()
model.layer1 = nn.DataParallel(model.layer1, device_ids=[0])
model.layer2 = nn.DataParallel(model.layer2, device_ids=[0])
model.layer3 = nn.DataParallel(model.layer3, device_ids=[0])
model.fc1 = nn.DataParallel(model.fc1, device_ids=[0])
model.fc2 = nn.DataParallel(model.fc2, device_ids=[0])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scaler = GradScaler()

In [10]:
def evaluate_accuracy(loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data[0].to(device), data[1].to(device).float().unsqueeze(1)
            outputs = model(images)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    model.train()
    return 100 * correct / total

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    for epoch in range(10):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(inputs)
            with autocast():
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0, norm_type=2)
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
        train_accuracy = evaluate_accuracy(train_loader, model, device)
        val_accuracy = evaluate_accuracy(val_loader, model, device)
        print(f'Epoch {epoch + 1}: Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')
with open("SvG_ProfileResults.txt", "w") as f:
    f.write(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

[Epoch 1, Batch 100] loss: 0.776
Epoch 1: Train Accuracy: 81.59%, Validation Accuracy: 80.40%
[Epoch 2, Batch 100] loss: 0.449
Epoch 2: Train Accuracy: 82.54%, Validation Accuracy: 81.55%
[Epoch 3, Batch 100] loss: 0.421
Epoch 3: Train Accuracy: 83.29%, Validation Accuracy: 82.25%
[Epoch 4, Batch 100] loss: 0.399
Epoch 4: Train Accuracy: 83.29%, Validation Accuracy: 82.33%
[Epoch 5, Batch 100] loss: 0.383
Epoch 5: Train Accuracy: 82.07%, Validation Accuracy: 80.70%
[Epoch 6, Batch 100] loss: 0.369
Epoch 6: Train Accuracy: 84.34%, Validation Accuracy: 83.50%
[Epoch 7, Batch 100] loss: 0.357
Epoch 7: Train Accuracy: 86.16%, Validation Accuracy: 84.17%
[Epoch 8, Batch 100] loss: 0.345
Epoch 8: Train Accuracy: 86.69%, Validation Accuracy: 85.28%
[Epoch 9, Batch 100] loss: 0.326
Epoch 9: Train Accuracy: 86.70%, Validation Accuracy: 85.62%
[Epoch 10, Batch 100] loss: 0.347
Epoch 10: Train Accuracy: 87.16%, Validation Accuracy: 85.28%


In [14]:
torch.save(model.state_dict(), 'D://SvGmodel.pth')